## SPARC-4 mini-pipeline tools
***
# <font color='darkblue'>Polarimetry time series</font> 

This notebook shows an example for obtaining a polarimetric time series

* Eder Martioli, LNA 20 Jun 2023

<div class="alert alert-block alert-warning">
<b>WARNING:</b> it requires running <b>sp4_polarimetry.ipynb</b> in advance, for the same channel and modes, to generate *_polar.fits reduced files
</div>

In [1]:
import os,sys
sys.path.append(os.path.dirname(os.getcwd()))
import sparc4_product_plots as s4plt
import sparc4_pipeline_lib as s4pipelib
import sparc4_utils as s4utils
import sparc4_db as s4db
import glob

/Users/eder/anaconda3/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


### User inputs

In [2]:
# set an object ID matching the ID in the image header keyword 'OBJECT'
OBJECTID = "HD111579"
OBJECT_SUFFIX = "hd111_l2"

# set night directory name
NIGHTDIR = '20230503'
# set raw data root directory
DATADIR =  "/Volumes/Samsung_T5/Data/SPARC4/minidata/"
# set reduced data root directory
REDUCEDDIR = "/Volumes/Samsung_T5/Data/SPARC4/minidata/reduced/"

# set SPARC4 channel
CHANNEL = 4  # 1, 2, 3 or 4

# whether or not to force reduction even if product already exists
FORCE = False

# get SPARC4 pipeline parameters
p = s4pipelib.init_s4_p(datadir=DATADIR,
                        reducedir=REDUCEDDIR,
                        nightdir=NIGHTDIR,
                        channels="{}".format(CHANNEL),
                        print_report=False)

# create database of raw data for reduction
db = s4db.create_db_from_observations(p['filelists'][CHANNEL-1], 
                                      p['DB_KEYS'], 
                                      include_img_statistics=p["INCLUDE_IMG_STATISTICS"], 
                                      include_only_fullframe=p["FULL_FRAMES_ONLY"], 
                                      output=p['s4db_files'][CHANNEL-1])

# detect all detector modes
detector_modes = s4db.get_detector_modes_observed(db, 
                                                  science_only=True,
                                                  detector_keys=p["DETECTOR_MODE_KEYWORDS"])
# get first valid key
mode_key = next(iter(detector_modes))

# set PHOTOMETRIC mode
inst_mode = p['INSTMODE_POLARIMETRY_KEYVALUE']

# we need the reduce directory 
reduce_ch_dir = p['reduce_directories'][CHANNEL-1]

# Calculate POLARIMETRY time series

In [3]:
# set wild card to identify polarimetric sequence
l2seq_wild_card = "_{}_polar.fits".format(OBJECT_SUFFIX)

# produce list of reduced polarimetry frames in the sequence
l2seq = glob.glob("{}/*{}".format(p['reduce_directories'][CHANNEL-1], l2seq_wild_card))

print(l2seq)

# set suffix for output time series filename
ts_suffix = "{}_s4c{}_{}_L4".format(NIGHTDIR,p['CHANNELS'][CHANNEL-1],OBJECTID.replace(" ",""))

# create polar time series product
polar_ts_L2product = s4pipelib.polar_time_series(l2seq,
                                                 reduce_dir=reduce_ch_dir,
                                                 ts_suffix=ts_suffix,
                                                 aperture_radius=p['APERTURE_RADIUS_FOR_PHOTOMETRY_IN_POLAR'],
                                                 min_aperture=p['MIN_APERTURE_FOR_POLARIMETRY'],
                                                 max_aperture=p['MAX_APERTURE_FOR_POLARIMETRY'],
                                                 force=FORCE)

['/Volumes/Samsung_T5/Data/SPARC4/minidata/reduced//sparc4acs4//20230503/20230503_s4c4_002119_hd111_l2_polar.fits']
Packing time series data for polar file 1 of 1


# Check the time series data in the FITS file

In [4]:
import astropy.io.fits as fits
# open the polarimetric time series product:
hdul = fits.open(polar_ts_L2product)
# print info, where each extension contains the time series data for one source. 
hdul.info()

Filename: /Volumes/Samsung_T5/Data/SPARC4/minidata/reduced//sparc4acs4//20230503/20230503_s4c4_HD111579_L4_ts.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      25   ()      
  1  POLAR_TIMESERIES    1 BinTableHDU     65   36R x 28C   [D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D]   
